# Libraries

In [1]:
pip install gower

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gower
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)

# Functions

In [4]:
def is_high_season(date):
    year = date.year
        #verano, asueto revolución, invierno, asueto constitución, asueto petróleo
    if (pd.Timestamp(year=year, month=7, day=17) <= date <= pd.Timestamp(year=year, month=8, day=27)) or \
       (pd.Timestamp(year=year, month=11, day=18) <= date <= pd.Timestamp(year=year, month=11, day=20)) or \
       (pd.Timestamp(year=year, month=12, day=18) <= date <= pd.Timestamp(year=year+1, month=1, day=5)) or \
       (pd.Timestamp(year=year, month=2, day=3) <= date <= pd.Timestamp(year=year, month=2, day=5)) or \
       (pd.Timestamp(year=year, month=3, day=16) <= date <= pd.Timestamp(year=year, month=3, day=18)) or \
        (pd.Timestamp(year=year, month=3, day=25) <= date <= pd.Timestamp(year=year, month=4, day=7)):
        return 1
    return 0

In [5]:
def remove_outliers(df, column_name, threshold=3):
    """
    Remove rows with outliers in a specific column of a DataFrame.
    
    Parameters:
        - df: pandas DataFrame
            The DataFrame containing the data.
        - column_name: str
            The name of the column to check for outliers.
        - threshold: int or float, optional (default=3)
            The number of standard deviations away from the mean to consider as an outlier.
    
    Returns:
        - pandas DataFrame
            The DataFrame with the outliers removed.
    """
    mean = np.mean(df[column_name])
    std = np.std(df[column_name])
    lower_bound = mean - threshold * std
    upper_bound = mean + threshold * std
    
    return df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

In [6]:
def find_dbscan_clusters(df_scaled, max_clusters, start_eps=0.1, end_eps=2.0, step_eps=0.1, min_samples=5):
    for eps in np.arange(start_eps, end_eps, step_eps):
        db = DBSCAN(eps=eps, min_samples=min_samples).fit(df_scaled)
        labels = db.labels_
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)  # Exclude noise label (-1)
        print(f"eps: {eps:.2f}, clusters: {n_clusters}")
        if n_clusters <= max_clusters:
            return labels, eps
    return None, None

# Limpieza de datos

In [7]:
df = pd.read_csv('C:\\Users\\axelm\\Downloads\\QueryGrosera_v4.csv', encoding='latin1')
df.head()

,Reserva,Tipo_Habitacion,Clasificacion_tipo_habitacion,Paquete,Canal,Agencia,Estatus_res,Capacidad_hotel,Numero_personas,Numero_adultos,Numero_men,Numero_noches,Numero_habitaciones,IngresoMto,FechaRegistro,FechaLlegada,FechaSalida
0,6576,ESTD C/BALCON SN12ASB,ASB,WALK IN ...,FAX ...,SUNWING VACATIONS ...,SALIDA,735,1,1,0,1,1,101.60,2019-12-10,2020-01-15,2020-01-16
1,82869,MV LUXURY 1K SN12GSU,GSU,WALK IN ...,MULTIVACACIONES 2,RESORT ADVANTAGE ...,SALIDA,735,2,2,0,4,1,109.44,2019-03-19,2019-06-16,2019-06-20
2,82868,MV LUXURY 2Q SN12GSU,GSU,WALK IN ...,MULTIVACACIONES 2,RESORT ADVANTAGE ...,SALIDA,735,3,2,0,4,1,109.44,2019-03-19,2019-06-16,2019-06-20
3,62469,JR SUITE 1K SN12MJS,MJS,NINGUNO ...,LADA 800 NACIONAL68 ...,RESORT ADVANTAGE ...,SALIDA,735,2,2,0,2,1,113.12,2019-02-22,2019-02-22,2019-02-24
4,63254,JR SUITE 1K SN12MJS,MJS,NINGUNO ...,MULTIVACACIONES 1,RESORT ADVANTAGE ...,SALIDA,735,2,2,0,2,1,113.70,2019-02-12,2019-02-13,2019-02-15


In [8]:
df.columns.tolist()

['Reserva',
 'Tipo_Habitacion',
 'Clasificacion_tipo_habitacion',
 'Paquete',
 'Canal',
 'Agencia',
 'Estatus_res',
 'Capacidad_hotel',
 'Numero_personas',
 'Numero_adultos',
 'Numero_men',
 'Numero_noches',
 'Numero_habitaciones',
 'IngresoMto',
 'FechaRegistro',
 'FechaLlegada',
 'FechaSalida']

In [9]:
df_cleaned = remove_outliers(df, "IngresoMto", threshold=3)

df_cleaned['FechaLlegada'] = pd.to_datetime(df_cleaned['FechaLlegada'])

df_cleaned['Tipo_temporada'] = df_cleaned['FechaLlegada'].apply(is_high_season)

df_cleaned['Diferencia_reservacion_llegada'] = pd.to_datetime(df_cleaned['FechaLlegada']) - pd.to_datetime(df_cleaned['FechaRegistro'])

df_cleaned.head()

C:\Users\axelm\AppData\Local\Temp\ipykernel_6260\3485774568.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['FechaLlegada'] = pd.to_datetime(df_cleaned['FechaLlegada'])
C:\Users\axelm\AppData\Local\Temp\ipykernel_6260\3485774568.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Tipo_temporada'] = df_cleaned['FechaLlegada'].apply(is_high_season)
C:\Users\axelm\AppData\Local\Temp\ipykernel_6260\3485774568.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

,Reserva,Tipo_Habitacion,Clasificacion_tipo_habitacion,Paquete,Canal,Agencia,Estatus_res,Capacidad_hotel,Numero_personas,Numero_adultos,Numero_men,Numero_noches,Numero_habitaciones,IngresoMto,FechaRegistro,FechaLlegada,FechaSalida,Tipo_temporada,Diferencia_reservacion_llegada
0,6576,ESTD C/BALCON SN12ASB,ASB,WALK IN ...,FAX ...,SUNWING VACATIONS ...,SALIDA,735,1,1,0,1,1,101.60,2019-12-10,2020-01-15,2020-01-16,0,36 days
1,82869,MV LUXURY 1K SN12GSU,GSU,WALK IN ...,MULTIVACACIONES 2,RESORT ADVANTAGE ...,SALIDA,735,2,2,0,4,1,109.44,2019-03-19,2019-06-16,2019-06-20,0,89 days
2,82868,MV LUXURY 2Q SN12GSU,GSU,WALK IN ...,MULTIVACACIONES 2,RESORT ADVANTAGE ...,SALIDA,735,3,2,0,4,1,109.44,2019-03-19,2019-06-16,2019-06-20,0,89 days
3,62469,JR SUITE 1K SN12MJS,MJS,NINGUNO ...,LADA 800 NACIONAL68 ...,RESORT ADVANTAGE ...,SALIDA,735,2,2,0,2,1,113.12,2019-02-22,2019-02-22,2019-02-24,0,0 days
4,63254,JR SUITE 1K SN12MJS,MJS,NINGUNO ...,MULTIVACACIONES 1,RESORT ADVANTAGE ...,SALIDA,735,2,2,0,2,1,113.70,2019-02-12,2019-02-13,2019-02-15,0,1 days


In [10]:
df_cleaned.to_csv('QueryGroseraLimpia.csv', index=False)

# Dummies

In [15]:
columns_to_use = ['Tipo_Habitacion', 'Clasificacion_tipo_habitacion', 'Paquete', 'Canal', 'Estatus_res', 'Capacidad_hotel', 'Numero_personas', 'Numero_adultos', 'Numero_noches', 'IngresoMto', 'Tipo_temporada']

df_dummies = df_cleaned[columns_to_use]

df_dummies = df_dummies.fillna(0)

df_dummies = df_dummies.replace([np.inf, -np.inf], 0)

# Preprocesamiento con OneHotEncoder para variables categóricas y StandardScaler para ingresos
categorical_features = ['Tipo_Habitacion', 'Clasificacion_tipo_habitacion', 'Paquete', 'Canal', 'Estatus_res', "Tipo_temporada"]
numeric_features = ['Capacidad_hotel', 'Numero_personas', 'Numero_adultos', 'Numero_noches', 'IngresoMto']

categorical_transformer = OneHotEncoder(handle_unknown='ignore')
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

df_dummies = preprocessor.fit_transform(df_dummies)

# Gower

In [14]:
# # define weights
# default_weight = 1
# ingreso_mto_weight = 5

# # initialize weight list
# weights = [default_weight] * df_dummies.shape[1]

# # Find the index of the 'IngresoMto' column and assign the specific weight
# ingreso_mto_index = df_dummies.columns.get_loc('IngresoMto')
# weights[ingreso_mto_index] = ingreso_mto_weight

In [15]:
# gower_distances = gower.gower_matrix(df_dummies, weight = weights)

In [16]:
# gower_df = pd.DataFrame(gower_distances, index=df.index, columns=df.index)

# DBSCAN

In [19]:
# scaler = StandardScaler()
# df_scaled = scaler.fit_transform(df_dummies)

In [20]:
# max_clusters = 8

# # Find suitable DBSCAN parameters
# labels, eps = find_dbscan_clusters(df_scaled, max_clusters)

eps: 0.10, clusters: 1991
eps: 0.20, clusters: 2061
eps: 0.30, clusters: 2172
eps: 0.40, clusters: 2235
eps: 0.50, clusters: 2195
eps: 0.60, clusters: 2146
eps: 0.70, clusters: 2110
eps: 0.80, clusters: 2098
eps: 0.90, clusters: 2043
eps: 1.00, clusters: 2020
eps: 1.10, clusters: 1986
eps: 1.20, clusters: 1874
eps: 1.30, clusters: 1866
eps: 1.40, clusters: 1894
eps: 1.50, clusters: 1874
eps: 1.60, clusters: 1734
eps: 1.70, clusters: 1642
eps: 1.80, clusters: 1614
eps: 1.90, clusters: 1581


In [21]:
# if labels is not None:
#     df_cleaned['Cluster'] = labels
#     print(f"Found solution with eps: {eps:.2f}")
# else:
#     print("Could not find a suitable eps value to achieve the desired number of clusters.")

# df_cleaned.head()

Could not find a suitable eps value to achieve the desired number of clusters.


,index,Reserva,Tipo_Habitacion,Clasificacion_tipo_habitacion,Paquete,Canal,Agencia,Estatus_res,Capacidad_hotel,Numero_personas,Numero_adultos,Numero_men,Numero_noches,Numero_habitaciones,IngresoMto,Tipo_temporada,Diferencia_reservacion_llegada
0,0,6576,ESTD C/BALCON SN12ASB,ASB,WALK IN ...,FAX ...,SUNWING VACATIONS ...,SALIDA,735,1,1,0,1,1,101.60,0,36 days
1,1,82869,MV LUXURY 1K SN12GSU,GSU,WALK IN ...,MULTIVACACIONES 2,RESORT ADVANTAGE ...,SALIDA,735,2,2,0,4,1,109.44,0,89 days
2,2,82868,MV LUXURY 2Q SN12GSU,GSU,WALK IN ...,MULTIVACACIONES 2,RESORT ADVANTAGE ...,SALIDA,735,3,2,0,4,1,109.44,0,89 days
3,3,62469,JR SUITE 1K SN12MJS,MJS,NINGUNO ...,LADA 800 NACIONAL68 ...,RESORT ADVANTAGE ...,SALIDA,735,2,2,0,2,1,113.12,0,0 days
4,4,63254,JR SUITE 1K SN12MJS,MJS,NINGUNO ...,MULTIVACACIONES 1,RESORT ADVANTAGE ...,SALIDA,735,2,2,0,2,1,113.70,0,1 days


In [16]:
db = DBSCAN(eps=0.5, min_samples=5).fit(df_dummies)

df_cleaned['Cluster'] = db.labels_

df_cleaned.head()

C:\Users\axelm\AppData\Local\Temp\ipykernel_6260\3300780122.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['Cluster'] = db.labels_


,Reserva,Tipo_Habitacion,Clasificacion_tipo_habitacion,Paquete,Canal,Agencia,Estatus_res,Capacidad_hotel,Numero_personas,Numero_adultos,Numero_men,Numero_noches,Numero_habitaciones,IngresoMto,FechaRegistro,FechaLlegada,FechaSalida,Tipo_temporada,Diferencia_reservacion_llegada,Cluster
0,6576,ESTD C/BALCON SN12ASB,ASB,WALK IN ...,FAX ...,SUNWING VACATIONS ...,SALIDA,735,1,1,0,1,1,101.60,2019-12-10,2020-01-15,2020-01-16,0,36 days,-1
1,82869,MV LUXURY 1K SN12GSU,GSU,WALK IN ...,MULTIVACACIONES 2,RESORT ADVANTAGE ...,SALIDA,735,2,2,0,4,1,109.44,2019-03-19,2019-06-16,2019-06-20,0,89 days,-1
2,82868,MV LUXURY 2Q SN12GSU,GSU,WALK IN ...,MULTIVACACIONES 2,RESORT ADVANTAGE ...,SALIDA,735,3,2,0,4,1,109.44,2019-03-19,2019-06-16,2019-06-20,0,89 days,-1
3,62469,JR SUITE 1K SN12MJS,MJS,NINGUNO ...,LADA 800 NACIONAL68 ...,RESORT ADVANTAGE ...,SALIDA,735,2,2,0,2,1,113.12,2019-02-22,2019-02-22,2019-02-24,0,0 days,-1
4,63254,JR SUITE 1K SN12MJS,MJS,NINGUNO ...,MULTIVACACIONES 1,RESORT ADVANTAGE ...,SALIDA,735,2,2,0,2,1,113.70,2019-02-12,2019-02-13,2019-02-15,0,1 days,-1


In [17]:
df_cleaned['Cluster'].value_counts()

-1       7745
 5       3231
 275     2977
 488     2974
 423     2837
 491     2513
 16      2114
 30      2062
 2       1940
 4       1805
 643     1269
 15      1179
 299     1161
 291      921
 331      909
 285      860
 585      852
 259      804
 584      785
 335      784
 394      768
 391      670
 110      655
 36       631
 451      600
 564      568
 364      548
 69       513
 339      501
 365      487
 720      441
 390      439
 273      426
 761      424
 73       399
 452      387
 546      371
 606      350
 429      344
 442      337
 489      337
 68       322
 612      315
 698      303
 673      281
 279      272
 363      267
 902      256
 545      250
 277      247
 687      241
 89       236
 1        234
 333      234
 309      233
 874      224
 567      223
 280      220
 381      207
 639      204
 733      204
 45       200
 565      194
 495      193
 879      188
 498      187
 281      183
 7        183
 34       182
 308      181
 91       180
 3    

In [18]:
df_cleaned['Cluster'].nunique()

1058